In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import ltn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# 1. Configuração de Ambiente
# Verifica se há uma GPU disponível para acelerar o treino.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# ==========================================
# 2. PREPARAÇÃO DOS DADOS (CIFAR-10)
# ==========================================

# Define as transformações: Converte imagem para Tensor e normaliza para o intervalo [-1, 1].
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Baixa o dataset CIFAR-10 completo.
print("Carregando dataset CIFAR-10...")
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# --- Filtragem de Classes ---
# O CIFAR-10 possui 10 classes. Para este trabalho de classificação binária,
# filtramos apenas Gatos (índice 3) e Cachorros (índice 5).
mask_cats_dogs = (torch.tensor(full_trainset.targets) == 3) | (torch.tensor(full_trainset.targets) == 5)

# Aplica o filtro aos targets (labels) e aos dados (imagens).
full_trainset.targets = torch.tensor(full_trainset.targets)[mask_cats_dogs]
full_trainset.data = full_trainset.data[mask_cats_dogs]

# Separação dos índices para criar dois "grupos" lógicos distintos.
# No LTN, não usamos labels (0 ou 1) diretamente no treino supervisionado clássico.
# Em vez disso, separamos os dados em variáveis lógicas: "grupo de gatos" e "grupo de cães".
cat_indices = (full_trainset.targets == 3)
dog_indices = (full_trainset.targets == 5)

# Processamento das imagens: Transposição de (H, W, C) para (C, H, W) e normalização para float [0, 1].
cats_data_raw = torch.tensor(full_trainset.data[cat_indices].transpose(0, 3, 1, 2)).float() / 255.0
dogs_data_raw = torch.tensor(full_trainset.data[dog_indices].transpose(0, 3, 1, 2)).float() / 255.0

# Normalização final para o intervalo [-1, 1] (padrão para redes neurais convergirem melhor).
cats_data = (cats_data_raw - 0.5) / 0.5
dogs_data = (dogs_data_raw - 0.5) / 0.5

# --- Configuração de Treino ---
# Limitamos a 500 amostras de cada classe para demonstração rápida no Colab.
# Para usar o dataset completo, basta remover o fatiamento [:500].
limit = 500
cats_data = cats_data[:limit].to(device)
dogs_data = dogs_data[:limit].to(device)
batch_size = 32

print(f"Imagens de gatos prontas: {cats_data.shape}")
print(f"Imagens de cães prontas: {dogs_data.shape}")

# ==========================================
# 3. MODELO NEURO-SIMBÓLICO (PREDICADO)
# ==========================================

class SimpleCNN(nn.Module):
    """
    Uma Rede Neural Convolucional (CNN) simples.
    No contexto do LTN, esta rede atua como o Predicado 'Dog(x)'.
    Entrada: Imagem (3x32x32).
    Saída: Valor real em [0, 1] representando o grau de verdade.
    """
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)       # Camada Convolucional 1
        self.pool = nn.MaxPool2d(2, 2)        # Max Pooling
        self.conv2 = nn.Conv2d(6, 16, 5)      # Camada Convolucional 2
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # Camada Densa 1
        self.fc2 = nn.Linear(120, 84)         # Camada Densa 2
        self.fc3 = nn.Linear(84, 1)           # Camada de Saída (1 neurónio)
        self.sigmoid = nn.Sigmoid()           # Função de ativação final (Crucial para LTN)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # Achata os tensores para a camada densa
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        # A Sigmoid garante que o resultado seja interpretável como probabilidade/verdade fuzzy [0,1]
        x = self.sigmoid(x)
        return x

# Instancia a rede e a define como um Predicado LTN
cnn = SimpleCNN().to(device)
Dog = ltn.Predicate(cnn)

# ==========================================
# 4. DEFINIÇÃO DA LÓGICA FUZZY
# ==========================================

# Conectivos e Quantificadores definidos conforme o artigo (Lógica do Produto e Reichenbach)
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard()) # Operador de Negação
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f") # Quantificador Universal
SatAgg = ltn.fuzzy_ops.SatAgg() # Agregador de Satisfação (calcula a "nota" final da KB)

# Otimizador padrão (Adam)
optimizer = optim.Adam(Dog.parameters(), lr=0.001)

# ==========================================
# 5. LOOP DE TREINAMENTO (Neuro-Simbólico)
# ==========================================
print("\n--- Iniciando Treinamento LTN ---")

n_epochs = 50
steps_per_epoch = len(cats_data) // batch_size

for epoch in range(n_epochs):
    epoch_loss = 0.0

    # Embaralha os índices a cada época para garantir aleatoriedade nos batches
    perm = torch.randperm(len(cats_data))

    for i in range(steps_per_epoch):
        # Seleciona os índices do batch atual
        idx = perm[i*batch_size : (i+1)*batch_size]

        # Cria os mini-batches de tensores
        batch_cats = cats_data[idx]
        batch_dogs = dogs_data[idx]

        optimizer.zero_grad()

        # --- PASSO A: Aterramento (Grounding) ---
        # Mapeia os dados brutos (imagens) para variáveis lógicas.
        var_dog = ltn.Variable("dog", batch_dogs) # Variável que representa "qualquer cachorro"
        var_cat = ltn.Variable("cat", batch_cats) # Variável que representa "qualquer gato"

        # --- PASSO B: Definição dos Axiomas (Base de Conhecimento) ---

        # Axioma 1: "Para todo x em var_dog, Dog(x) é Verdadeiro".
        # Ensinamos a rede que as imagens do grupo 'dogs' devem retornar 1.
        axiom_dog = Forall(var_dog, Dog(var_dog))

        # Axioma 2: "Para todo x em var_cat, NÃO Dog(x) é Verdadeiro".
        # Ensinamos a rede que as imagens do grupo 'cats' devem retornar 0 (negativo de Dog).
        axiom_cat = Forall(var_cat, Not(Dog(var_cat)))

        # --- PASSO C: Cálculo da Satisfação e Perda ---
        # Agrega a satisfação de todos os axiomas. Quanto mais próximo de 1.0, melhor.
        sat = SatAgg(axiom_dog, axiom_cat)

        # A Loss é o inverso da satisfação. Queremos Loss -> 0.
        loss = 1. - sat

        # --- PASSO D: Backpropagation ---
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Exibe o progresso a cada 5 épocas
    if epoch % 5 == 0:
        print(f"Época {epoch:02d}: Loss Média = {epoch_loss / steps_per_epoch:.4f} | Satisfação Estimada = {1 - (epoch_loss / steps_per_epoch):.4f}")

print("Treino concluído!")

# ==========================================
# 6. VALIDAÇÃO / TESTE
# ==========================================
print("\n--- Teste de Validação Final ---")
with torch.no_grad():
    # Pega uma amostra de cada classe para teste manual
    sample_dog = dogs_data[0:1]
    sample_cat = cats_data[0:1]

    # Consulta o Predicado (a CNN treinada)
    score_dog = Dog(ltn.Variable("x", sample_dog)).value.item()
    score_cat = Dog(ltn.Variable("x", sample_cat)).value.item()

    print(f"Imagem de Cachorro -> Predicado Dog(x): {score_dog:.4f} (Ideal: > 0.9)")
    print(f"Imagem de Gato     -> Predicado Dog(x): {score_cat:.4f} (Ideal: < 0.1)")

Usando dispositivo: cpu
Carregando dataset CIFAR-10...
Imagens de gatos prontas: torch.Size([500, 3, 32, 32])
Imagens de cães prontas: torch.Size([500, 3, 32, 32])

--- Iniciando Treinamento LTN ---
Época 00: Loss Média = 0.4985 | Satisfação Estimada = 0.5015
Época 05: Loss Média = 0.4436 | Satisfação Estimada = 0.5564
Época 10: Loss Média = 0.3842 | Satisfação Estimada = 0.6158
Época 15: Loss Média = 0.3076 | Satisfação Estimada = 0.6924
Época 20: Loss Média = 0.2235 | Satisfação Estimada = 0.7765
Época 25: Loss Média = 0.1731 | Satisfação Estimada = 0.8269
Época 30: Loss Média = 0.1363 | Satisfação Estimada = 0.8637
Época 35: Loss Média = 0.1550 | Satisfação Estimada = 0.8450
Época 40: Loss Média = 0.1391 | Satisfação Estimada = 0.8609
Época 45: Loss Média = 0.1314 | Satisfação Estimada = 0.8686
Treino concluído!

--- Teste de Validação Final ---
Imagem de Cachorro -> Predicado Dog(x): 0.9687 (Ideal: > 0.9)
Imagem de Gato     -> Predicado Dog(x): 0.0260 (Ideal: < 0.1)
